In [2]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pickle.load(open('../data/dataset_level1.pkl', 'rb'))

### Features required
- batting_team
- bowling_team
- city
- current_score
- ball_left 
- wickets_left
- current_rr
- last_five_over_runs

In [4]:
df.isna().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8549
venue                  0
dtype: int64

In [5]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: count, dtype: int64

In [6]:
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x: x[0]), df['city'])

In [7]:
df['city'] = cities

In [8]:
df.isna().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [9]:
df.drop(columns=['venue'], inplace=True)

In [10]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
625,7,Pakistan,New Zealand,0.1,1,0,Dunedin
626,7,Pakistan,New Zealand,0.2,0,0,Dunedin
627,7,Pakistan,New Zealand,0.3,0,0,Dunedin
628,7,Pakistan,New Zealand,0.4,0,Hassan Nawaz,Dunedin
629,7,Pakistan,New Zealand,0.5,1,0,Dunedin
...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,0,Auckland
325418,2690,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland
325419,2690,Pakistan,New Zealand,19.4,2,0,Auckland
325420,2690,Pakistan,New Zealand,19.5,1,0,Auckland


In [11]:
# Considering only those cities where minimum of 5 matches are played i.e. roughly 600 balls
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [14]:
eligible_cities

['Colombo',
 'Dubai',
 'Johannesburg',
 'Auckland',
 'Mirpur',
 'Sydney',
 'Lahore',
 'Cape Town',
 'Dhaka',
 'London',
 'Durban',
 'Wellington',
 'Melbourne',
 'Pallekele',
 'Christchurch',
 'Barbados',
 'Centurion',
 'Abu Dhabi',
 'Lauderhill',
 'Mount Maunganui',
 'Southampton',
 'Hamilton',
 'Gros Islet',
 'Manchester',
 'Nottingham',
 'St Lucia',
 'Karachi',
 'Kolkata',
 'Bridgetown',
 'Cardiff',
 'Mumbai',
 'Adelaide',
 'Tarouba',
 'Birmingham',
 'Brisbane',
 "St George's",
 'Kandy',
 'Perth',
 'Sharjah',
 'Chittagong',
 'Delhi',
 'Ahmedabad',
 'Providence',
 'Chandigarh',
 'Kingston',
 'Rajkot',
 'Napier',
 'Nagpur',
 'Pune',
 'Bangalore',
 'Sylhet',
 'Dambulla',
 'Hobart',
 'Trinidad']

In [33]:
df = df[df['city'].isin(eligible_cities)]

In [34]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
1044,11,West Indies,India,0.1,0,0,Lauderhill
1045,11,West Indies,India,0.2,0,JD Campbell,Lauderhill
1046,11,West Indies,India,0.3,0,0,Lauderhill
1047,11,West Indies,India,0.4,1,0,Lauderhill
1048,11,West Indies,India,0.5,0,0,Lauderhill
...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,0,Auckland
325418,2690,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland
325419,2690,Pakistan,New Zealand,19.4,2,0,Auckland
325420,2690,Pakistan,New Zealand,19.5,1,0,Auckland


In [35]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [36]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
1044,11,West Indies,India,0.1,0,0,Lauderhill,0
1045,11,West Indies,India,0.2,0,JD Campbell,Lauderhill,0
1046,11,West Indies,India,0.3,0,0,Lauderhill,0
1047,11,West Indies,India,0.4,1,0,Lauderhill,1
1048,11,West Indies,India,0.5,0,0,Lauderhill,1
...,...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,0,Auckland,149
325418,2690,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,149
325419,2690,Pakistan,New Zealand,19.4,2,0,Auckland,151
325420,2690,Pakistan,New Zealand,19.5,1,0,Auckland,152


In [37]:
df['over'] = df['ball'].apply(lambda x: str(x).split('.')[0])
df['ball_no'] = df['ball'].apply(lambda x: str(x).split('.')[1])

In [38]:
df['balls_bowled'] = (df['over'].astype(int)*6) + df['ball_no'].astype(int)

In [39]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x: 0 if x<0 else x)

In [40]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
1044,11,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119
1045,11,West Indies,India,0.2,0,JD Campbell,Lauderhill,0,0,2,2,118
1046,11,West Indies,India,0.3,0,0,Lauderhill,0,0,3,3,117
1047,11,West Indies,India,0.4,1,0,Lauderhill,1,0,4,4,116
1048,11,West Indies,India,0.5,0,0,Lauderhill,1,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,0,Auckland,149,19,2,116,4
325418,2690,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,149,19,3,117,3
325419,2690,Pakistan,New Zealand,19.4,2,0,Auckland,151,19,4,118,2
325420,2690,Pakistan,New Zealand,19.5,1,0,Auckland,152,19,5,119,1


In [41]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: x if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype(int)
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [42]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
1044,11,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10
1045,11,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9
1046,11,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9
1047,11,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9
1048,11,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2
325418,2690,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1
325419,2690,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1
325420,2690,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1


In [47]:
df['crr'] = (df['current_score'] * 6)/df['balls_bowled']

In [48]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
1044,11,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10,0.000000
1045,11,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9,0.000000
1046,11,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9,0.000000
1047,11,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9,1.500000
1048,11,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9,1.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2,7.706897
325418,2690,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1,7.641026
325419,2690,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1,7.677966
325420,2690,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1,7.663866


In [49]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30)['runs'].sum().values.tolist())

In [58]:
df['last_five'] = last_five

In [59]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
1044,11,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10,0.000000,NaN
1045,11,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9,0.000000,NaN
1046,11,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9,0.000000,NaN
1047,11,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9,1.500000,NaN
1048,11,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325417,2690,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2,7.706897,56.0
325418,2690,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1,7.641026,50.0
325419,2690,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1,7.677966,52.0
325420,2690,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1,7.663866,53.0


In [64]:
final_df = df.groupby('match_id')['runs'].sum().reset_index(name= 'final_score').merge(df, on= 'match_id')

In [66]:
final_df

,match_id,final_score,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,11,95,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10,0.000000,NaN
1,11,95,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9,0.000000,NaN
2,11,95,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9,0.000000,NaN
3,11,95,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9,1.500000,NaN
4,11,95,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84880,2690,153,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2,7.706897,56.0
84881,2690,153,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1,7.641026,50.0
84882,2690,153,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1,7.677966,52.0
84883,2690,153,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1,7.663866,53.0


In [68]:
final_df = final_df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five', 'final_score']]

In [69]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,final_score
0,West Indies,India,Lauderhill,0,119,10,0.000000,NaN,95
1,West Indies,India,Lauderhill,0,118,9,0.000000,NaN,95
2,West Indies,India,Lauderhill,0,117,9,0.000000,NaN,95
3,West Indies,India,Lauderhill,1,116,9,1.500000,NaN,95
4,West Indies,India,Lauderhill,1,115,9,1.200000,NaN,95
...,...,...,...,...,...,...,...,...,...
84880,Pakistan,New Zealand,Auckland,149,4,2,7.706897,56.0,153
84881,Pakistan,New Zealand,Auckland,149,3,1,7.641026,50.0,153
84882,Pakistan,New Zealand,Auckland,151,2,1,7.677966,52.0,153
84883,Pakistan,New Zealand,Auckland,152,1,1,7.663866,53.0,153


In [70]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        20199
final_score          0
dtype: int64

In [71]:
final_df.dropna(inplace=True)

In [73]:
final_df.isna().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
final_score      0
dtype: int64

In [75]:
final_df = final_df.sample(final_df.shape[0])

In [ ]:
pickle.dump(final_df, open('../data/final_data.pkl', 'wb'))